In [35]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

In [36]:
import numpy as np
import os, time

# 准备数据

In [37]:
text = open('./economic.txt', 'rb').read().decode(encoding='utf-8')
text[:250]

'沸腾了！ GDP首次突破100万亿，人均收入十年翻一番！芯片股大爆发！千亿资金狂买这类股，新抱团来了？！券商高呼：价值重估！\r\n芯片股大爆发，中国GDP首次突破100万亿元！\r\n截至收盘，沪指涨0.70%，报收3591点；深成指涨1.31%，报收15228点；创业板指涨1.84%，报收3184点。\r\n我国GDP首次突破100万亿元。\r\n2020年，我国国民经济在一季度下降后逐季恢复，如何评价全年经济运行表现？1月18日，在国新办就2020年国民经济运行情况举行的发布会上，国家统计局局长宁吉喆表示'

In [38]:
text

'沸腾了！ GDP首次突破100万亿，人均收入十年翻一番！芯片股大爆发！千亿资金狂买这类股，新抱团来了？！券商高呼：价值重估！\r\n芯片股大爆发，中国GDP首次突破100万亿元！\r\n截至收盘，沪指涨0.70%，报收3591点；深成指涨1.31%，报收15228点；创业板指涨1.84%，报收3184点。\r\n我国GDP首次突破100万亿元。\r\n2020年，我国国民经济在一季度下降后逐季恢复，如何评价全年经济运行表现？1月18日，在国新办就2020年国民经济运行情况举行的发布会上，国家统计局局长宁吉喆表示，我国二季度GDP增长3.2%，由负转正。三季度增长4.9%，四季度增长6.5%，全年经济增长2.3%，预计将成为全球唯一实现经济正增长的主要经济体。\r\n会上通报，初步核算，2020年国内生产总值1015986亿元，按可比价格计算，比上年增长2.3%。\r\n分季度看，一季度同比下降6.8%，二季度增长3.2%，三季度增长4.9%，四季度增长6.5%。\r\n分产业看，第一产业增加值77754亿元，比上年增长3.0%；第二产业增加值384255亿元，增长2.6%；第三产业增加值553977亿元，增长2.1%。\r\n2020年全国居民人均可支配收入32189元\r\n比2010年翻一番\r\n国家统计局今日发布了2020年我国居民收入和消费支出情况，2020年全国居民人均可支配收入为32189元，比2010年翻一番。具体情况如下：\r\n一、居民收入情况\r\n2020年，全国居民人均可支配收入32189元，比上年名义增长4.7%，扣除价格因素，实际增长2.1%。其中，城镇居民人均可支配收入43834元，增长（以下如无特别说明，均为同比名义增速）3.5%，扣除价格因素，实际增长1.2%；农村居民人均可支配收入17131元，增长6.9%，扣除价格因素，实际增长3.8%。\r\n2020年，全国居民人均可支配收入中位数27540元，增长3.8%，中位数是平均数的85.6%。其中，城镇居民人均可支配收入中位数40378元，增长2.9%，是平均数的92.1%；农村居民人均可支配收入中位数15204元，增长5.7%，是平均数的88.7%。\r\n图1 2020年全国居民人均可支配收入平均数与中位数\r\n按收入来源分，2020年，全国居民人均工资性收入17917元

In [39]:
vocab = sorted(set(text))
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab))

chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True)

In [40]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [41]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(138411,), dtype=int64, numpy=array([1112, 1498,  190, ...,  573,    5,    3], dtype=int64)>

In [42]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
seq_length = 14
# examples_per_epoch = len(text)//(seq_length+1)
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True) # 定义每个样本的数据

In [43]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [44]:
dataset = sequences.map(split_input_target)
# for input_example, target_example in dataset.take(1):
#     print("Input: ", input_example.numpy())
#     print("Target: ", target_example.numpy())

# 创建训练的batch

In [45]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 14), (64, 14)), types: (tf.int64, tf.int64)>

# 创建以单个字为基准的模型

In [46]:
vocab_size = len(vocab) # 字典大小
embedding_dim = 256 # 嵌入层维度
rnn_units = 1024 # 神经元数量


class nlg_for_fun(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True, 
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else: 
            return x

In [47]:
model = nlg_for_fun(
    vocab_size = len(ids_from_chars.get_vocabulary()),
    embedding_dim = embedding_dim,
    rnn_units = rnn_units
)

# 训练

In [24]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)

# 配置checkpoints
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

EPOCHS = 100

if os.path.exists('training_checkpoints/'):
    model.load_weights('training_checkpoints/ckpt')
    model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
else:
    model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/2
45/45 [==============================] - 215s 5s/step - loss: 3.1790
Epoch 2/2
45/45 [==============================] - 217s 5s/step - loss: 2.9944


# 预测

In [48]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature=temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "" or "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(['','[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float('inf')]*len(skip_ids),
            indices = skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())]) 
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits] 
        predicted_logits, states =  self.model(inputs=input_ids, states=states, 
                                              return_state=True)
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature
        # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [52]:
import random

random.randint(100, 140)

133

In [53]:
# 加载本地模型
model.load_weights('training_checkpoints/ckpt')
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

start = time.time()
states = None
next_char = tf.constant(['今日'])
result = [next_char]

import random
text_length = random.randint(100, 140)
for n in range(text_length):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)

end = time.time()

print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

print(f"\nRun time: {end - start}")

今日仍然较和环境的开采调
，但我们确认中小微实现了我析）在
WIPO 基本保持了一种非常如此相关活动的工具，并希望的横滨欧
环境和质量
就业大力务中心咨询支援课　　●强化市民和消费活动
资源区不确定量 

________________________________________________________________________________

Run time: 1.013993740081787
